## Evaluación Final Módulo 3

In [43]:
# Nos importamos todas las librerías que necesitaremos
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [44]:
# Cargamos los csv con los datos y vemos las primeras líneas (también se podría ver el final o un sample)
df_vuelos = pd.read_csv("Customer Flight Activity.csv", index_col=0)

df_vuelos.head()

,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
Loyalty Number,,,,,,,,,
100018,2017,1,3,0,3,1521,152.0,0,0
100102,2017,1,10,4,14,2030,203.0,0,0
100140,2017,1,6,0,6,1200,120.0,0,0
100214,2017,1,0,0,0,0,0.0,0,0
100272,2017,1,0,0,0,0,0.0,0,0


## 1. Exploración y análisis

In [45]:
df_clientes = pd.read_csv("Customer Loyalty History.csv")

df_clientes.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [46]:
#Sacamos la info para una exploración básica general: columnas, cantidad de filas, de nulos y el tipo de dato en cada columna.
df_vuelos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405624 entries, 100018 to 999986
Data columns (total 9 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Year                         405624 non-null  int64  
 1   Month                        405624 non-null  int64  
 2   Flights Booked               405624 non-null  int64  
 3   Flights with Companions      405624 non-null  int64  
 4   Total Flights                405624 non-null  int64  
 5   Distance                     405624 non-null  int64  
 6   Points Accumulated           405624 non-null  float64
 7   Points Redeemed              405624 non-null  int64  
 8   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 30.9 MB


In [47]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Loyalty Number      16737 non-null  int64  
 1   Country             16737 non-null  object 
 2   Province            16737 non-null  object 
 3   City                16737 non-null  object 
 4   Postal Code         16737 non-null  object 
 5   Gender              16737 non-null  object 
 6   Education           16737 non-null  object 
 7   Salary              12499 non-null  float64
 8   Marital Status      16737 non-null  object 
 9   Loyalty Card        16737 non-null  object 
 10  CLV                 16737 non-null  float64
 11  Enrollment Type     16737 non-null  object 
 12  Enrollment Year     16737 non-null  int64  
 13  Enrollment Month    16737 non-null  int64  
 14  Cancellation Year   2067 non-null   float64
 15  Cancellation Month  2067 non-null   float64
dtypes: f

In [48]:
# Vemos si hay nulos y dónde están
df_clientes.isnull().sum()

Loyalty Number            0
Country                   0
Province                  0
City                      0
Postal Code               0
Gender                    0
Education                 0
Salary                 4238
Marital Status            0
Loyalty Card              0
CLV                       0
Enrollment Type           0
Enrollment Year           0
Enrollment Month          0
Cancellation Year     14670
Cancellation Month    14670
dtype: int64

In [49]:
df_vuelos.isnull().sum()

Year                           0
Month                          0
Flights Booked                 0
Flights with Companions        0
Total Flights                  0
Distance                       0
Points Accumulated             0
Points Redeemed                0
Dollar Cost Points Redeemed    0
dtype: int64

In [50]:
# Comprobamos si es posible que los nulos de Salary sean porque aún estén estudiando o no trabajen
nulos_salary = df_clientes[df_clientes['Salary'].isnull()]
college_education = nulos_salary['Education'].unique()

print("Niveles educativos asociados a nulos en 'Salary':", college_education)

Niveles educativos asociados a nulos en 'Salary': ['College']


In [78]:
# Calculamos el porcentaje de nulos en Salary, para ver cómo de importante es y qué podemos hacer con él.
porcentaje_nulos_salary = df_clientes["Salary"].isnull().mean() * 100
print(f"El porcentaje de nulos en Salary es del {porcentaje_nulos_salary.round(2)}%")

El porcentaje de nulos en Salary es del 25.32%


Es un porcentaje muy alto para cambiar por la media o la mediana, además coincide con que todos son de college, por lo que las regresiones o vecinos no encajarían, puede ser un error en la creación de la base de datos. Consultar con la empresa.

In [52]:
# Identificamos los valores únicos en todas las columnas para detectar inconsistencias o errores tipográficos.
for columna in df_vuelos:
    print(f'La columna {columna} tiene estos valores únicos:')
    print(df_vuelos[columna].unique())
    print('----------------------------') 

La columna Year tiene estos valores únicos:
[2017 2018]
----------------------------
La columna Month tiene estos valores únicos:
[ 1  9  2  3 11  4  5  7  6  8 10 12]
----------------------------
La columna Flights Booked tiene estos valores únicos:
[ 3 10  6  0  8 11  9  4  7  5  2  1 12 13 14 16 15 17 18 19 20 21]
----------------------------
La columna Flights with Companions tiene estos valores únicos:
[ 0  4  7  1  6  3  5  2 10  8  9 11]
----------------------------
La columna Total Flights tiene estos valores únicos:
[ 3 14  6  0 15 11 12 10  8  9  7  5 16  2  1 17 13 22  4 19 18 21 26 20
 23 25 27 24 28 30 29 31 32]
----------------------------
La columna Distance tiene estos valores únicos:
[1521 2030 1200 ... 1217  617 4135]
----------------------------
La columna Points Accumulated tiene estos valores únicos:
[152.   203.   120.   ...  18.75 601.   626.  ]
----------------------------
La columna Points Redeemed tiene estos valores únicos:
[  0 341 364 310 445 312 343 366 38

In [53]:
# Como en dos columnas no se ven todos los datos únicos, buscamos si hay valores negativos

for columna in df_vuelos.select_dtypes(include=['int64', 'float64']).columns:
    negativos = df_vuelos[df_vuelos[columna] < 0]
    if not negativos.empty:
        print(f"\nColumna: '{columna}'")
        print(f"Cantidad de valores negativos: {len(negativos)}")
        print(negativos)
    else:
        print(f"\nColumna: '{columna}' - No hay valores negativos")


Columna: 'Year' - No hay valores negativos

Columna: 'Month' - No hay valores negativos

Columna: 'Flights Booked' - No hay valores negativos

Columna: 'Flights with Companions' - No hay valores negativos

Columna: 'Total Flights' - No hay valores negativos

Columna: 'Distance' - No hay valores negativos

Columna: 'Points Accumulated' - No hay valores negativos

Columna: 'Points Redeemed' - No hay valores negativos

Columna: 'Dollar Cost Points Redeemed' - No hay valores negativos


In [54]:
# Hacemos lo mismo con el otro df
for columna in df_clientes:
    print(f'La columna {columna} tiene estos valores únicos:')
    print(df_clientes[columna].unique())
    print('----------------------------') 

La columna Loyalty Number tiene estos valores únicos:
[480934 549612 429460 ... 776187 906428 652627]
----------------------------
La columna Country tiene estos valores únicos:
['Canada']
----------------------------
La columna Province tiene estos valores únicos:
['Ontario' 'Alberta' 'British Columbia' 'Quebec' 'Yukon' 'New Brunswick'
 'Manitoba' 'Nova Scotia' 'Saskatchewan' 'Newfoundland'
 'Prince Edward Island']
----------------------------
La columna City tiene estos valores únicos:
['Toronto' 'Edmonton' 'Vancouver' 'Hull' 'Whitehorse' 'Trenton' 'Montreal'
 'Dawson Creek' 'Quebec City' 'Fredericton' 'Ottawa' 'Tremblant' 'Calgary'
 'Thunder Bay' 'Whistler' 'Peace River' 'Winnipeg' 'Sudbury'
 'West Vancouver' 'Halifax' 'London' 'Regina' 'Kelowna' "St. John's"
 'Victoria' 'Kingston' 'Banff' 'Moncton' 'Charlottetown']
----------------------------
La columna Postal Code tiene estos valores únicos:
['M2Z 4K1' 'T3G 6Y6' 'V6E 3D9' 'P1W 1K4' 'J8Y 3Z5' 'Y2K 6R0' 'P5S 6R4'
 'K8V 4B2' 'H2Y 2W

In [55]:
# Buscamos los valores negativos, pero SOLO en las columnas numéricas, para eso hacemos una variable 
columnas_numericas_clientes = df_clientes.select_dtypes(include=['int64', 'float64']).columns

for columna in columnas_numericas_clientes:
    negativos = df_clientes[df_clientes[columna] < 0]
    if not negativos.empty:
        print(f"OJO columna: {columna}")
        print(f"Cantidad de valores negativos: {len(negativos)}")
    else:
        print(f"Columna: {columna} - No hay valores negativos")

Columna: Loyalty Number - No hay valores negativos
OJO columna: Salary
Cantidad de valores negativos: 20
Columna: CLV - No hay valores negativos
Columna: Enrollment Year - No hay valores negativos
Columna: Enrollment Month - No hay valores negativos
Columna: Cancellation Year - No hay valores negativos
Columna: Cancellation Month - No hay valores negativos


In [56]:
# Vemos en detalle los negativos de salary
df_clientes[df_clientes['Salary'] < 0][['Country', 'Province', 'Salary']]

,Country,Province,Salary
1082,Canada,Quebec,-49830.0
1894,Canada,British Columbia,-12497.0
2471,Canada,British Columbia,-46683.0
3575,Canada,British Columbia,-45962.0
3932,Canada,British Columbia,-19325.0
4712,Canada,British Columbia,-43234.0
6560,Canada,Newfoundland,-10605.0
6570,Canada,Newfoundland,-17534.0
7373,Canada,Ontario,-58486.0
8576,Canada,Ontario,-31911.0


In [57]:
# Verificamos si hay duplicados
df_clientes.duplicated().sum()

0

In [58]:
df_vuelos.duplicated().sum() # Aquí hay duplicados pero entendemos que cada cliente vuela varias veces, por eso hay varios registros. Hay que comprobarlo.

228705

## 2. Limpieza y cambios

In [59]:
# Cambiamos las columnas float a int.
df_vuelos['Points Accumulated'] = df_vuelos['Points Accumulated'].astype(int)
df_vuelos['Points Accumulated'].dtypes

dtype('int32')

In [60]:
df_clientes['Cancellation Year'] = df_clientes['Cancellation Year'].astype('Int64')  # Int64 para mantener los NaN, ya que habíamos visto que aquí había
df_clientes['Cancellation Month'] = df_clientes['Cancellation Month'].astype('Int64')

print(df_clientes[['Cancellation Year', 'Cancellation Month']].dtypes)

Cancellation Year     Int64
Cancellation Month    Int64
dtype: object


In [61]:
# Cambiamos los valores negativos a positivos de Salary.
df_clientes['Salary'] = df_clientes['Salary'].abs()


In [62]:
# Comprobamos los duplicados que tenemos en vuelo, pero OJO hay que ver si son duplicados porque los clientes cogen diferentes vuelos o porque realmente
# coinciden todas las columnas
df_vuelos.duplicated().sum()

228705

In [63]:
# Tenemos que resetear el index para que cuente esa columna también
df_vuelos.reset_index(inplace=True)

In [64]:
# Verificamos los duplicados exactos en TODAS las columnas
duplicados_exactos = df_vuelos[df_vuelos.duplicated(keep=False)]  # keep=False marca todos los duplicados
print(f"Número de filas duplicadas (exactas): {len(duplicados_exactos)}")

Número de filas duplicadas (exactas): 3712


In [65]:
# Hay que eliminar las filas que son duplicadas EN TODAS LAS COLUMNAS: los mismos valores en todas las columnas, es decir, son diferentes vuelos.
df_vuelos = df_vuelos.drop_duplicates()
print(f'Los duplicados que quedan son: {df_vuelos.duplicated().sum()}')

Los duplicados que quedan son: 0


In [66]:
# Creamos una columna nueva para los clientes activos
df_clientes['Is_Active'] = df_clientes['Cancellation Year'].isna()

# Verificar conteo
print(df_clientes['Is_Active'].value_counts())

Is_Active
True     14670
False     2067
Name: count, dtype: int64


In [67]:
# Comprobamos que se ha creado la columna
df_clientes.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month,Is_Active
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,<NA>,<NA>,True
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,<NA>,<NA>,True
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018,1,False
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,<NA>,<NA>,True
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,<NA>,<NA>,True


In [68]:
# Tenemos que juntar los dos dataframes, pero antes vamos a sacar de nuevo todos los datos y métricas.
display(df_vuelos.shape)
display(df_clientes.shape)

(403760, 10)

(16737, 17)

In [69]:
# No hay datos categóricos en vuelos
df_vuelos.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,403760.0,549875.38,258961.51,100018.0,326699.0,550598.0,772152.0,999986.0
Year,403760.0,2017.50,0.50,2017.0,2017.0,2018.0,2018.0,2018.0
Month,403760.0,6.50,3.45,1.0,4.0,7.0,10.0,12.0
Flights Booked,403760.0,4.13,5.23,0.0,0.0,1.0,8.0,21.0
Flights with Companions,403760.0,1.04,2.08,0.0,0.0,0.0,1.0,11.0
Total Flights,403760.0,5.17,6.53,0.0,0.0,1.0,10.0,32.0
Distance,403760.0,1214.46,1434.10,0.0,0.0,525.0,2342.0,6293.0
Points Accumulated,403760.0,124.24,146.68,0.0,0.0,53.0,240.0,676.0
Points Redeemed,403760.0,30.84,125.76,0.0,0.0,0.0,0.0,876.0
Dollar Cost Points Redeemed,403760.0,2.50,10.17,0.0,0.0,0.0,0.0,71.0


In [70]:
df_clientes.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,16737.0,549735.880445,258912.132453,100018.0,326603.0,550434.0,772019.0,999986.0
Salary,12499.0,79359.340907,34749.691464,9081.0,59246.5,73455.0,88517.5,407228.0
CLV,16737.0,7988.896536,6860.98228,1898.01,3980.84,5780.18,8940.58,83325.38
Enrollment Year,16737.0,2015.253211,1.979111,2012.0,2014.0,2015.0,2017.0,2018.0
Enrollment Month,16737.0,6.669116,3.398958,1.0,4.0,7.0,10.0,12.0
Cancellation Year,2067.0,2016.503145,1.380743,2013.0,2016.0,2017.0,2018.0,2018.0
Cancellation Month,2067.0,6.962748,3.455297,1.0,4.0,7.0,10.0,12.0


In [71]:
df_clientes.describe(include = "object").T

,count,unique,top,freq
Country,16737,1,Canada,16737
Province,16737,11,Ontario,5404
City,16737,29,Toronto,3351
Postal Code,16737,55,V6E 3D9,911
Gender,16737,2,Female,8410
Education,16737,5,Bachelor,10475
Marital Status,16737,3,Married,9735
Loyalty Card,16737,3,Star,7637
Enrollment Type,16737,2,Standard,15766


In [72]:
# Juntamos los dos dataframes con todos los datos por la columna Loyalty Number, usando outer para no perder ningún dato

datos_completos = pd.merge(df_clientes, df_vuelos, on='Loyalty Number', how='outer')
datos_completos.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,...,Is_Active,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,...,True,2017,1,3,0,3,1521,152,0,0
1,100018,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,...,True,2017,2,2,2,4,1320,132,0,0
2,100018,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,...,True,2018,10,6,4,10,3110,311,385,31
3,100018,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,...,True,2017,4,4,0,4,924,92,0,0
4,100018,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,...,True,2017,5,0,0,0,0,0,0,0


In [73]:
# Comprobamos los datos del nuevo df
datos_completos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403760 entries, 0 to 403759
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               403760 non-null  int64  
 1   Country                      403760 non-null  object 
 2   Province                     403760 non-null  object 
 3   City                         403760 non-null  object 
 4   Postal Code                  403760 non-null  object 
 5   Gender                       403760 non-null  object 
 6   Education                    403760 non-null  object 
 7   Salary                       301500 non-null  float64
 8   Marital Status               403760 non-null  object 
 9   Loyalty Card                 403760 non-null  object 
 10  CLV                          403760 non-null  float64
 11  Enrollment Type              403760 non-null  object 
 12  Enrollment Year              403760 non-null  int64  
 13 

In [74]:
datos_completos.columns

Index(['Loyalty Number', 'Country', 'Province', 'City', 'Postal Code',
       'Gender', 'Education', 'Salary', 'Marital Status', 'Loyalty Card',
       'CLV', 'Enrollment Type', 'Enrollment Year', 'Enrollment Month',
       'Cancellation Year', 'Cancellation Month', 'Is_Active', 'Year', 'Month',
       'Flights Booked', 'Flights with Companions', 'Total Flights',
       'Distance', 'Points Accumulated', 'Points Redeemed',
       'Dollar Cost Points Redeemed'],
      dtype='object')

In [75]:
datos_completos.tail()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,...,Is_Active,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
403755,999986,Canada,Ontario,Ottawa,K1F 2R2,Female,Bachelor,46594.0,Married,Nova,...,True,2018,7,15,0,15,1995,199,0,0
403756,999986,Canada,Ontario,Ottawa,K1F 2R2,Female,Bachelor,46594.0,Married,Nova,...,True,2018,9,0,0,0,0,0,0,0
403757,999986,Canada,Ontario,Ottawa,K1F 2R2,Female,Bachelor,46594.0,Married,Nova,...,True,2018,10,6,0,6,708,70,0,0
403758,999986,Canada,Ontario,Ottawa,K1F 2R2,Female,Bachelor,46594.0,Married,Nova,...,True,2018,11,11,4,15,2040,204,0,0
403759,999986,Canada,Ontario,Ottawa,K1F 2R2,Female,Bachelor,46594.0,Married,Nova,...,True,2018,12,0,0,0,0,0,0,0


In [76]:
# Guardamos el nuevo dataframe como csv
datos_completos.to_csv("archivos_datos/datos_completos.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'archivos_datos'